In [165]:
import pandas as pd
from pandas import json_normalize
import json
from sklearn.model_selection import train_test_split

In [166]:
dementia_data = pd.read_csv("dementia.csv")

In [167]:
dementia_data.describe(include="all")
dementia_data

,File Path,Transcription
0,adrso024,There's a cookie jar and the lid is off the c...
1,adrso025,"Tell me everything that's going on. Well, the..."
2,adrso027,There's a picture. There's a little girl and ...
3,adrso028,How she would find her. And the mother's wish...
4,adrso031,"What do you see going on? Well, the boy's on ..."
...,...,...
82,adrso247,"Oh, little boy's getting in the cookies. He's..."
83,adrso248,"Now the girl is washing dishes, the water is ..."
84,adrso249,Okay. There she's washing the dishes and ther...
85,adrso250,Where do you mean in the picture? Here's the ...


In [168]:
dementia_data["Classification"] = "Dementia"

In [169]:
dementia_data

,File Path,Transcription,Classification
0,adrso024,There's a cookie jar and the lid is off the c...,Dementia
1,adrso025,"Tell me everything that's going on. Well, the...",Dementia
2,adrso027,There's a picture. There's a little girl and ...,Dementia
3,adrso028,How she would find her. And the mother's wish...,Dementia
4,adrso031,"What do you see going on? Well, the boy's on ...",Dementia
...,...,...,...
82,adrso247,"Oh, little boy's getting in the cookies. He's...",Dementia
83,adrso248,"Now the girl is washing dishes, the water is ...",Dementia
84,adrso249,Okay. There she's washing the dishes and ther...,Dementia
85,adrso250,Where do you mean in the picture? Here's the ...,Dementia


In [170]:
dementia_data.drop("File Path", axis=1, inplace=True)

In [171]:
non_dementia_data = pd.read_csv("no_dementia.csv")

In [172]:
non_dementia_data.describe(include="all")

,File Path,Transcription
count,79,79
unique,79,79
top,adrso018,Just look at this one and tell me everything ...
freq,1,1


In [173]:
non_dementia_data

,File Path,Transcription
0,adrso018,Just look at this one and tell me everything ...
1,adrso019,"Okay, little boy is on a stool that looks lik..."
2,adrso003,"Starting from the left, I see two children st..."
3,adrso017,"All right, we have a few more tasks to go thr..."
4,adrso005,"Oh my, such a mess, the sink is overflowing. ..."
...,...,...
74,adrso267,"going on in the picture. Okay, there it is. Y..."
75,adrso263,"Okay, there's the picture. Everything is in t..."
76,adrso280,There's the picture. This little boy has his ...
77,adrso270,The mother's drying the dish and the water's ...


In [174]:
non_dementia_data["Classification"] = "No Dementia"

In [175]:
non_dementia_data.drop("File Path", axis=1, inplace=True)

In [176]:
dementia_data.to_csv("prepared_dementia.csv",index=False)
non_dementia_data.to_csv("prepared_non_dementia.csv",index=False)

In [177]:
prepared_dementia_data = pd.read_csv("prepared_dementia.csv")

In [178]:
prepared_non_dementia_data = pd.read_csv("prepared_non_dementia.csv")

In [179]:
prepared_dementia_data

,Transcription,Classification
0,There's a cookie jar and the lid is off the c...,Dementia
1,"Tell me everything that's going on. Well, the...",Dementia
2,There's a picture. There's a little girl and ...,Dementia
3,How she would find her. And the mother's wish...,Dementia
4,"What do you see going on? Well, the boy's on ...",Dementia
...,...,...
82,"Oh, little boy's getting in the cookies. He's...",Dementia
83,"Now the girl is washing dishes, the water is ...",Dementia
84,Okay. There she's washing the dishes and ther...,Dementia
85,Where do you mean in the picture? Here's the ...,Dementia


In [180]:
prepared_non_dementia_data

,Transcription,Classification
0,Just look at this one and tell me everything ...,No Dementia
1,"Okay, little boy is on a stool that looks lik...",No Dementia
2,"Starting from the left, I see two children st...",No Dementia
3,"All right, we have a few more tasks to go thr...",No Dementia
4,"Oh my, such a mess, the sink is overflowing. ...",No Dementia
...,...,...
74,"going on in the picture. Okay, there it is. Y...",No Dementia
75,"Okay, there's the picture. Everything is in t...",No Dementia
76,There's the picture. This little boy has his ...,No Dementia
77,The mother's drying the dish and the water's ...,No Dementia


In [181]:
completed_data = pd.merge(prepared_dementia_data, prepared_non_dementia_data, on=['Transcription', 'Classification'], how='outer')

In [185]:
train_data, validate_data = train_test_split(completed_data, test_size=0.2, random_state=42)
def row_to_jsonl(row):
    return json.dumps({
        "messages": [
            {"role": "system", "content": "Transcript: " + row['Transcription']},
            {"role": "assistant", "content": row['Classification']}
        ]
    })
    
train_jsonl_string = "\n".join(train_data.apply(row_to_jsonl, axis=1))
validate_jsonl_string = "\n".join(validate_data.apply(row_to_jsonl, axis=1))

In [186]:
with open('train_dementia_data.jsonl', 'w') as f:
    f.write(train_jsonl_string)

In [187]:
with open('validate_dementia_data.jsonl', 'w') as f:
    f.write(validate_jsonl_string)